# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


C:\Users\ClayMason313\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [14]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [17]:
# frequency table for prestige and whether or not someone was admitted
print (pd.crosstab(df['admit'], df['prestige'], rownames=['admit']))

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12



## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [18]:
dummy_prestige = pd.get_dummies(df['prestige'],prefix='prestige')
dummy_prestige.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: not sure

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [19]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_prestige.ix[:, 'prestige_1':])
handCalc.head()

C:\Users\ClayMason313\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [20]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
print (pd.crosstab(handCalc['admit'], handCalc['prestige_1.0'], rownames=['admit']))

prestige_1.0    0   1
admit                
0             243  28
1              93  33


In [21]:
print(handCalc)


     admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  \
0        0  380.0  3.61             0             0             1   
1        1  660.0  3.67             0             0             1   
2        1  800.0  4.00             1             0             0   
3        1  640.0  3.19             0             0             0   
4        0  520.0  2.93             0             0             0   
5        1  760.0  3.00             0             1             0   
6        1  560.0  2.98             1             0             0   
7        0  400.0  3.08             0             1             0   
8        1  540.0  3.39             0             0             1   
9        0  700.0  3.92             0             1             0   
10       0  800.0  4.00             0             0             0   
11       0  440.0  3.22             1             0             0   
12       1  760.0  4.00             1             0             0   
13       0  700.0  3.08           

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [26]:
odds_exposed = (33/(33+28))
odds_exposed
#Cancel first calc
odds_exposed = 33.0/28
odds_exposed

1.1785714285714286

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [28]:
odds_unexposed = (93/(243+93))
odds_unexposed
#Cancel first calc

odds_unexposed = (93/243)
                  
odds_unexposed

0.38271604938271603

#### 3.3 Calculate the odds ratio

In [29]:
OR = odds_exposed/odds_unexposed
OR

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of being accepted into college from a top ranked college are three times as high as a lower ranked school. 

#### 3.5 Print the cross tab for prestige_4

In [30]:
print (pd.crosstab(handCalc['admit'], handCalc['prestige_4.0'], rownames=['admit']))

prestige_4.0    0   1
admit                
0             216  55
1             114  12


#### 3.6 Calculate the OR 

In [31]:
OR = (12*216.0)/(114*55)
print (1-OR) 

0.5866028708133971


#### 3.7 Write this finding in a sentence

Answer: the lowest tiering of schools has  a 60% less chance of getting into school. 

## Part 4. Analysis

In [32]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_prestige.ix[:, 'prestige_2':])
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [33]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [34]:
train_cols = data.columns[1:]

#### 4.2 Fit the model

In [35]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [36]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 09 Jan 2018   Pseudo R-squ.:                 0.08166
Time:                        01:20:44   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [37]:
np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

In [38]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
np.exp(conf)

,2.5%,97.5%,OR
gre,1.000074,1.004372,1.002221
gpa,1.136120,4.183113,2.180027
prestige_2.0,0.272168,0.942767,0.506548
prestige_3.0,0.133377,0.515419,0.262192
prestige_4.0,0.093329,0.479411,0.211525
intercept,0.002207,0.194440,0.020716


#### 4.5 Interpret the OR of Prestige_2

In [39]:
data.describe()


,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
count,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.0
mean,0.317380,587.858942,3.392242,0.372796,0.304786,0.168766,1.0
std,0.466044,115.717787,0.380208,0.484159,0.460898,0.375017,0.0
min,0.000000,220.000000,2.260000,0.000000,0.000000,0.000000,1.0
25%,0.000000,520.000000,3.130000,0.000000,0.000000,0.000000,1.0
50%,0.000000,580.000000,3.400000,0.000000,0.000000,0.000000,1.0
75%,1.000000,660.000000,3.670000,1.000000,1.000000,0.000000,1.0
max,1.000000,800.000000,4.000000,1.000000,1.000000,1.000000,1.0


#### 4.6 Interpret the OR of GPA

Answer: not sure. Don't think they are twice as likely to get in. Prestige column had binary output, so I wouldn't think my interpretation would be the same. 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [49]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)
        
    m = int(n / arrays[0].size)
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in np.arange(1, arrays[0].size):
            out[int(j*m):int((j+1))*int(m),1:] = out[0:int(m),1:]
    return out


In [50]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print (gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print(gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [51]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])

C:\Users\ClayMason313\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


#### 5.2 Make predictions on the enumerated dataset

In [53]:
combos['admit_pred'] = result.predict(combos[train_cols])

combos.tail()
len(combos)

400

#### 5.3 Interpret findings for the last 4 observations

Answer:Not sure that I'm following this exercise. we are testing different combinations of the variables to predict acceptance, but I'm  not sure how to interpret the output. We have 400 lines of data now. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.